In [1]:
import tensorflow as tf
source = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz'
model_dir = tf.keras.utils.get_file(fname='ssd_mobilenet_v2_320x320_coco17_tpu-8',origin = source, untar=True,cache_dir = './235-SSD/model_1')

2023-03-12 10:19:52.324019: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-12 10:19:52.324229: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
model = tf.saved_model.load(model_dir+'/saved_model')

2023-03-12 10:20:06.314110: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-12 10:20:06.314659: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-12 10:20:06.315216: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Magnus): /proc/driver/nvidia/version does not exist
2023-03-12 10:20:06.318710: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
signature = model.signatures['serving_default']

In [4]:
import numpy as np
import cv2
# Load an image from a file
image = cv2.imread('Screenshot 2023-03-10 190018(1).jpg')

# Convert the image from BGR to RGB format
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Resize the image to 320x320 pixels
image = cv2.resize(image, (320, 320))

# Convert the image to a float32 tensor
image = tf.convert_to_tensor(image, dtype=tf.uint8)

# Add a batch dimension to the tensor
image = tf.expand_dims(image, axis=0)

output_tensor = signature(image)


In [5]:
import matplotlib.pyplot as plt
box = output_tensor['detection_boxes']
classes = output_tensor['detection_classes']
scores = output_tensor['detection_scores']
num_detections = output_tensor['num_detections']
with open('Cat_labels.txt') as f:

    lines = f.readlines()
    lines  = [line.strip() for line in lines]
    
    




In [6]:
image = cv2.imread('Screenshot 2023-03-10 190018(1).jpg')
confidence_threshold = 0.6
for i in range(int(num_detections[0])):
    if scores[0][i] > confidence_threshold:
        boxes = box[0][i]
        x1 = int(boxes[1] * image.shape[1])
        y1 = int(boxes[0] * image.shape[0])
        x2 = int(boxes[3] * image.shape[1])
        y2 = int(boxes[2] * image.shape[0])
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, lines[int(classes[0][i])-1], (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
cv2.imshow('image', image)
cv2.waitKey(0)

        



225